In [1]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline
import shutil
import os
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.decomposition import RandomizedPCA
from sklearn.cross_validation import train_test_split
import numpy as np
#from PIL import Image
#from PyImagePlot.montage_manager import *

/usr/lib64/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df = pd.read_csv("../features/rural-ResNet50-pool.csv")
print df.shape

(71448, 2049)


In [3]:
df.head()

,image_paths,pool5.0,pool5.1,pool5.2,pool5.3,pool5.4,pool5.5,pool5.6,pool5.7,pool5.8,...,pool5.2038,pool5.2039,pool5.2040,pool5.2041,pool5.2042,pool5.2043,pool5.2044,pool5.2045,pool5.2046,pool5.2047
0,/jupyter-nfs/data/Liberia/rural/southeast_tile...,0.137256,0.033329,0.243107,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.147094,0.000000,0.0
1,/jupyter-nfs/data/Liberia/rural/southeast_tile...,1.413542,0.016365,0.089910,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.284178,0.0,0.0,0.068741,0.0,0.0,0.0,0.000000,0.022701,0.0
2,/jupyter-nfs/data/Liberia/rural/southeast_tile...,0.106881,0.000000,0.070017,0.026601,0.009001,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.001180,0.0,0.0,0.0,0.132630,0.000000,0.0
3,/jupyter-nfs/data/Liberia/rural/southeast_tile...,0.405045,0.009956,0.000000,0.000000,0.045935,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.015846,0.000000,0.0
4,/jupyter-nfs/data/Liberia/rural/southeast_tile...,0.279609,0.000000,0.409207,0.000000,0.021985,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.003470,0.000000,0.0


In [4]:
images = df.iloc[:,0]

In [5]:
X = np.array(df.iloc[:,1:])


In [8]:
## samples look random enough so just split data into two
N = int(df.shape[0]*.15)
X_train = X[:N,:] 
X_test = X[N:,:]


In [ ]:
km = KMeans(n_clusters=100, init='k-means++',n_init=500, max_iter=5000, random_state=0, n_jobs=-4)
km.fit(X_train)

In [ ]:
import collections

def make_counter_df(input_list):
    counters = collections.Counter(input_list)
    counters_df = pd.DataFrame.from_dict(counters, orient = 'index')
    return counters_df/counters_df.sum()

In [ ]:
def organize_images(kmeans_imgs, features, image_paths):
    cluster_orders = RandomizedPCA(n_components=1).fit_transform(kmeans_imgs.cluster_centers_)
    clusters = km.transform(features)
    clusters_distances_ordered = clusters[:,list(np.argsort(list(cluster_orders), axis = 0).squeeze())]
    clusters_df = pd.DataFrame({"image_paths": image_paths, 'cluster': np.argmin(clusters_distances_ordered, axis = 1), 
        'dist': np.min(clusters_distances_ordered*10, axis = 1).astype(int)})
    
    return clusters_df

In [ ]:
clustered_train_ordered = organize_images(km, X_train, df.iloc[:N,0])
clustered_test_ordered = organize_images(km, X_test, df.iloc[N:,0])

In [ ]:
clustered_train_ordered.head()

In [ ]:
make_counter_df(clustered_train_ordered.cluster).plot()


In [ ]:
make_counter_df(clustered_test_ordered.cluster).plot()

In [ ]:
clustered_train_ordered[["image_paths","cluster","dist"]].to_csv("../results/100_cluster_distributions_rural_train.csv", 
                                                                    index = False)
clustered_test_ordered[["image_paths","cluster","dist"]].to_csv("../results/100_cluster_distributions_rural_test.csv", 
                                                                    index = False)
